# Rag From Scratch: Indexing

![Screenshot 2024-03-25 at 8.23.02 PM.png](indexing.png)

## Set Environment Vars and API Keys

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQQ_API_KEY")

## Part 12: Multi-representation Indexing

Flow:

 ![Screenshot 2024-03-16 at 5.54.55 PM.png](multiindexing.png)

Docs:

https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector

- đọc 2 bài trên 2 trang web để làm docs
- thực hiện tóm tắt 2 bài
- lưu trữ vào không gian chromaDB, docs sẽ được ánh xạ đến một doc_id và được embedding thông qua hf_embeddings
- tìm kiếm bằng similarity

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://medium.com/@pankaj_pandey/introduction-to-retrieval-augmented-generation-rag-9209bf8a076d")
docs = loader.load()

loader = WebBaseLoader("https://medium.com/humansdotai/an-introduction-to-ai-agents-e8c4afd2ee8f")
docs.extend(loader.load())

In [ ]:
# !pip install langchain beautifulsoup4 requests

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://cloud.google.com/use-cases/retrieval-augmented-generation")
docs = loader.load()

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Retrieval-augmented_generation")
docs.extend(loader.load())

In [26]:
print(docs)
for doc in docs:
    print(doc.metadata)
    print(doc.page_content)

new_docs = []
for doc in docs:
    new_doc = doc
    new_doc.page_content = doc.page_content[:1000]
    new_docs.append(new_doc)

[Document(metadata={'source': 'https://cloud.google.com/use-cases/retrieval-augmented-generation', 'title': 'What is Retrieval-Augmented Generation (RAG)? | Google Cloud', 'description': 'Retrieval-augmented generation (RAG) combines LLMs with external knowledge bases to improve their outputs. Learn more with Google Cloud.', 'language': 'en-US'}, page_content='What is Retrieval-Augmented Generation (RAG)? | Google CloudPage ContentsTopicsRAGWhat is Retrieval-Augmented Generation (RAG)?RAG (Retrieval-Augmented Generation) is an AI framework that combines the strengths of traditional information retrieval systems (such as search and databases) with the capabilities of generative large language models (LLMs). By combining your data and world knowledge with LLM language skills, grounded generation is more accurate, up-to-date, and relevant to your specific needs. Check out this e-book to unlock your “Enterprise Truth.”Get started for free35:30Grounding for Gemini with Vertex AI Search and 

In [27]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatGroq()
    | StrOutputParser()
)

summaries = chain.batch(new_docs, {"max_concurrency": 1})

In [28]:
summaries

['Retrieval-Augmented Generation (RAG) is an AI framework that combines traditional information retrieval systems with generative large language models. RAG enhances the accuracy, timeliness, and relevance of AI-generated content by integrating data and world knowledge with language skills. The RAG process involves retrieving and pre-processing relevant information from external data sources, such as web pages, knowledge bases, and databases. This information is then utilized to generate more grounded and specific outputs for users. To learn more and get started for free, you can access the e-book "Unlock your \'Enterprise Truth\'" and explore Google Cloud\'s Vertex AI Search and DIY RAG options.',
 'Retrieval-augmented generation is a method of generating responses to queries that involves first retrieving relevant documents from a large corpus of text, and then using those documents to inform the response generation. The process involves four main steps: indexing, retrieval, augmenta

In [31]:
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=hf_embeddings)

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [30]:
# !pip install chromadb

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached wrapt-1.17.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached websockets-14.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached cachetools-5.5.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 

In [32]:
doc_ids

['6d570b08-c111-47b5-a195-5d4f60265c72',
 '8c09b199-5afc-47d9-821c-616687d94d60']

In [33]:
retriever.docstore.mget(doc_ids)

[Document(metadata={'source': 'https://cloud.google.com/use-cases/retrieval-augmented-generation', 'title': 'What is Retrieval-Augmented Generation (RAG)? | Google Cloud', 'description': 'Retrieval-augmented generation (RAG) combines LLMs with external knowledge bases to improve their outputs. Learn more with Google Cloud.', 'language': 'en-US'}, page_content='What is Retrieval-Augmented Generation (RAG)? | Google CloudPage ContentsTopicsRAGWhat is Retrieval-Augmented Generation (RAG)?RAG (Retrieval-Augmented Generation) is an AI framework that combines the strengths of traditional information retrieval systems (such as search and databases) with the capabilities of generative large language models (LLMs). By combining your data and world knowledge with LLM language skills, grounded generation is more accurate, up-to-date, and relevant to your specific needs. Check out this e-book to unlock your “Enterprise Truth.”Get started for free35:30Grounding for Gemini with Vertex AI Search and 

In [34]:
query = "What is agent"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(metadata={'doc_id': '6d570b08-c111-47b5-a195-5d4f60265c72'}, page_content='Retrieval-Augmented Generation (RAG) is an AI framework that combines traditional information retrieval systems with generative large language models. RAG enhances the accuracy, timeliness, and relevance of AI-generated content by integrating data and world knowledge with language skills. The RAG process involves retrieving and pre-processing relevant information from external data sources, such as web pages, knowledge bases, and databases. This information is then utilized to generate more grounded and specific outputs for users. To learn more and get started for free, you can access the e-book "Unlock your \'Enterprise Truth\'" and explore Google Cloud\'s Vertex AI Search and DIY RAG options.')

In [35]:
retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]

/tmp/ipykernel_1601037/3791815623.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'What is Retrieval-Augmented Generation (RAG)? | Google CloudPage ContentsTopicsRAGWhat is Retrieval-Augmented Generation (RAG)?RAG (Retrieval-Augmented Generation) is an AI framework that combines the strengths of traditional information retrieval systems (such as search and databases) with the capabilities of generative large language models (LLMs). By combining your data and world knowledge with LLM language skills, grounded generation is more accurate, up-to-date, and relevant to your specifi'

## Part 13: ColBERT

RAGatouille makes it as simple to use ColBERT.

ColBERT generates a contextually influenced vector for each token in the passages.

ColBERT similarly generates vectors for each token in the query.

Then, the score of each document is the sum of the maximum similarity of each query embedding to any of the document embeddings:

See [here](https://hackernoon.com/how-colbert-helps-developers-overcome-the-limits-of-rag) and [here](https://python.langchain.com/docs/integrations/retrievers/ragatouille) and [here](https://til.simonwillison.net/llms/colbert-ragatouille).

- sử dụng colbert là hệ RAG pretrained model luôn
- sử dụng index() để indexing tạo bộ database
- sử dụng search() để retrieval
- sử dụng invoke() để thực hiện trả về output từ input

In [36]:
!pip install ragatouille

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Using cached ujson-5.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.3 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached llama_index_indices_managed_llama_cloud-0.6.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached jiter-0.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.8 M

In [37]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/home/trung/.conda/envs/python310/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [38]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.

    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"] if "extract" in page else None

full_document = get_wikipedia_page("Document_retrieval")

In [ ]:
RAG.index(
    collection=[full_document],
    index_name="Doc-1",
    max_document_length=180,
    split_documents=True,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Feb 06, 10:16:00] #> Creating directory .ragatouille/colbert/indexes/Doc-1 


#> Starting...
#> Starting...
nranks = 2 	 num_gpus = 2 	 device=0
nranks = 2 	 num_gpus = 2 	 device=1


/home/trung/.conda/envs/python310/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/home/trung/.conda/envs/python310/lib/python3.10/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/home/trung/.conda/envs/python310/lib/python3.10/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/home/trung/.conda/envs/python310/lib/python3.10/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', 

[Feb 06, 10:16:06] [1] 		 #> Encoding 3 passages..
[Feb 06, 10:16:06] [0] 		 #> Encoding 4 passages..
[Feb 06, 10:16:08] [0] 		 avg_doclen_est = 114.16667175292969 	 len(local_sample) = 4
[Feb 06, 10:16:08] [1] 		 avg_doclen_est = 114.16667175292969 	 len(local_sample) = 3
[Feb 06, 10:16:08] [0] 		 Creating 256 partitions.
[Feb 06, 10:16:08] [0] 		 *Estimated* 799 embeddings.
[Feb 06, 10:16:08] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Doc-1/plan.json ..
Clustering 775 points in 128D to 256 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
  Iteration 19 (0.03 s, search 0.02 s): objective=108.294 imbalance=1.456 nsplit=0       
[Feb 06, 10:16:08] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/trung/.conda/envs/python310/lib/python3.10/site-packages/colbert/indexing/collection_indexer.py:256: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sub_sample = torch.l

In [5]:
results = RAG.search(query="What is an example for form based indexing?", k=3)
results

Loading searcher for index Doc-1 for the first time... This may take a few seconds
[Jul 06, 00:08:11] #> Loading codec...
[Jul 06, 00:08:11] #> Loading IVF...
[Jul 06, 00:08:11] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 06, 00:08:48] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3279.36it/s]

[Jul 06, 00:08:48] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 130.10it/s]

[Jul 06, 00:08:48] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Jul 06, 00:09:20] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is an example for form based indexing?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 2054, 2003, 2019, 2742, 2005, 2433, 2241, 5950, 2075, 1029,
         102,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'content': '== Variations ==\nThere are two main classes of indexing schemata for document retrieval systems: form based (or word based), and content based indexing. The document classification scheme (or indexing algorithm) in use determines the nature of the document retrieval system.\n\n\n=== Form based ===\nForm based document retrieval addresses the exact syntactic properties of a text, comparable to substring matching in string searches. The text is generally unstructured and not necessarily in a natural language, the system could for example be used to process large sets of chemical representations in molecular biology. A suffix tree algorithm is an example for form based indexing.',
  'score': 25.978090286254883,
  'rank': 1,
  'document_id': '83decb83-f58e-4d89-b9c1-51f09daadcfa',
  'passage_id': 2},
 {'content': '== Example: PubMed ==\nThe PubMed form interface features the "related articles" search which works through a comparison of words from the documents\' title, abstr

In [6]:
retriever = RAG.as_langchain_retriever(k=3)
retriever.invoke("What is an example for form based indexing?")

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[Document(page_content='== Variations ==\nThere are two main classes of indexing schemata for document retrieval systems: form based (or word based), and content based indexing. The document classification scheme (or indexing algorithm) in use determines the nature of the document retrieval system.\n\n\n=== Form based ===\nForm based document retrieval addresses the exact syntactic properties of a text, comparable to substring matching in string searches. The text is generally unstructured and not necessarily in a natural language, the system could for example be used to process large sets of chemical representations in molecular biology. A suffix tree algorithm is an example for form based indexing.'),
 Document(page_content='== Example: PubMed ==\nThe PubMed form interface features the "related articles" search which works through a comparison of words from the documents\' title, abstract, and MeSH terms using a word-weighted algorithm.\n\n\n== See also ==\nCompound term processing\n